## Coder un aglorithme du simplexe

Nous allons partir de la donnée d'un problème, sous forme matricielle et dans sa forme canonique, définir une variante de l'algorithme du simplexe et l'exécuter sur notre problème, pour obtenir 0, une ou des solutions optimales au problème.

Prenons l'exemple de notre problème de chocolaterie.

Dans sa forme canonique, le programme linéaire s'écrit ainsi :

$
\begin{array}{lcrcrcrcrcrcl}
  \text{maximiser :} &  & x_1 & + & 6 \cdot x_2 & + & 13 \cdot x_3 &&\\[2mm]
  \text{tel que :} &  & x_1 & + & x_2 & + & x_3 & \leqslant & 400 \\
  &  & &  & x_2 & + & 3 \cdot x_3 & \leqslant & 600 \\
  &  & &  &  &  & x1 & \leqslant & 200 \\
  &  & &  &  &  & x_2 & \leqslant & 300 \\
 &  &  & & & & x_1,x_2,x_3 & \geqslant & 0
\end{array}
$


Pour exécuter des programmes, toute forme se rapprochant de tableau(x), de quelque dimension que ce soit, est appréciable. On préférera donc regarder le problème dans sa forme matricielle (équivalente) :

$\begin{array}{lcl}
\text{maximiser :} & &  ^{t}c\times X\\
\text{tel que :} & & A \times X \leqslant b\\
\text{où :} & & c = \begin{pmatrix}1\\6\\13\end{pmatrix}\\
&& A = \begin{pmatrix}1&1&1\\0&1&3\\1&0&0\\0&1&0\end{pmatrix}\\
&& b = \begin{pmatrix}400\\600\\200\\300\end{pmatrix}\\
\end{array}$


Cette forme ressemble plus à quelque chose qui sera facilement manipulable. Nous pouvons ainsi représenter notre problème au moyen de 3 variables :

In [67]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
# Les chocolats:
c = [1, 6, 13]
A = [[1, 1, 1],
     [0, 1, 3],
     [1, 0, 0],
     [0, 1, 0]]
b = [400, 600, 200, 300]

Quelques remarques techniques :

- Notons qu'en guise de matrice, nous créerons en python une liste de listes `t` et considérerons l'élément `t[i][j]` comme l'élément situé en ligne `i-1` et colonne `j-1` de la matrice, puisqu'usuellement, les indices commencent à 1 dans les matrices en maths, mais à 0 dans les listes en informatique.
- Nous n'allons donc pas utiliser de type plus complexe (et plus correct) pour les matrices (nous aurions pu par exemple utiliser les types de `numpy`, qui sont très répandus). Seulement les types de base de python que sont les listes (et donc les listes de listes).
- Cependant, à dessein de représentation graphique, pour suivre l'évolution de vos matrices/*tableaux*, vous pouvez utiliser :

In [69]:
from sympy import Matrix, pprint
pprint(Matrix(A))

⎡1  1  1⎤
⎢       ⎥
⎢0  1  3⎥
⎢       ⎥
⎢1  0  0⎥
⎢       ⎥
⎣0  1  0⎦


Les bases étant posées, nous allons fabriquer une fonction `simplex` qui étant donnés `A`, `b` et `c` retourne la valeur de l'objectif max, et les valeurs des variables qui réalisent cet optimum.

### Résumé de l'algorithme du simplexe

#### Pré-requis

Tout d'abord, nous supposons qu'il existe une solution nulle au programme linéaire (nous avons vu en CM comment passer d'un programme linéaire qui n'en admet pas à un programme linéaire qui en admet une, grâce au *programme auxiliaire*).

Pour le problème de la chocolaterie, nous sommes dans ce cas, puisque
$A \times \begin{pmatrix}0\\0\\0\end{pmatrix} \leqslant b$.
En effet, on a $\begin{pmatrix}0\\0\\0\\0\end{pmatrix} \leqslant \begin{pmatrix}400\\600\\200\\300\end{pmatrix}$.

En python :

In [70]:
y_nul = [0, 0, 0, 0]
for l, y_el in enumerate(y_nul):
    assert y_el <= b[l], f"Contrainte #{l} non respectée"

#### Plan d'action

Nous allons suivre le même raffinage (la même décomposition du problème) que celui vu en cours et en TD. Il est rappelé ci-dessous.

On part d’une solution non-optimale du programme linéaire en forme standard.

Tant que l’objectif peut être amélioré, on évolue vers une solution proche qui améliore la fonction objectif. À chaque étape, on modifie la formulation du programme linéaire 

1. Choisir une variable non-basique à faire entrer dans la base
1. Choisir une variable basique à faire sortir de la base
1. Opérer le changement de base en faisant pivoter le tableau
1. Mettre à jour la base

### Création du tableau

Pour que le problème soit plus facile à manipuler informatiquement, on crée un *tableau* (terminologie utilisée historiquement pour l'algorithme du simplexe) qui contient
- les informations relatives au problème (programme linéaire)
- des informations relatives à la convergence du simplexe

Pour le problème de la chocolaterie, on aimerait créer initialement le *tableau* suivant :

$\begin{array}{c}
\\x_3 \\
x_4 \\
x_5 \\
x_6 \\
z
\end{array}
\begin{bmatrix}
\begin{array}{ccccccc|c}
x_0 &x_1 &x_2 &x_3 &x_4 &x_5 &x_6 &b \\ \hline
1 &1 &1 &1 &0 &0 &0 &400 \\
0 &1 &3 &0 &1 &0 &0 &600 \\
  1 &0 &0 &0 &0 &1 &0 &200 \\
0 &1 &0 &0 &0 &0 &1 &300 \\
\hline
 1 &6 &13 &0 &0 &0 &0 &0 \\
\end{array}
 \end{bmatrix}$

Il n'est pas très commode de garder les en-têtes des lignes et des colonnes en même temps que les valeurs numériques dans la matrice (quoiqu'on pourrait sans doute s'en sortir avec le module `pandas`), donc nous allons stocker d'une part les éléments numériques du *tableau*, et d'autre part les informations sur les en-têtes.

Pour les éléments numériques, nous allons utiliser une liste de listes :

In [71]:
tab = [[1, 1, 1, 1, 0, 0, 0, 400],
[0, 1, 3, 0, 1, 0, 0, 600],
[1, 0, 0, 0, 0, 1, 0, 200],
[0, 1, 0, 0, 0, 0, 1, 300],
[1, 6, 13,0, 0, 0, 0, 0]
]

In [72]:
print(tab[1][7])

600


Pour les en-têtes, les en-têtes de colonne ne vont pas bouger, donc il n'est pas nécessaire de les stocker. L'en-tête de la dernière ligne ne changera pas non plus, elle concernera toujours la fonction objectif. Par contre, les en-têtes des lignes précédentes risquent de changer : il faut les stocker. On peut par exemple ne stocker que la liste des indices des variables basiques. Ici :

In [73]:
[3, 4, 5, 6]

[3, 4, 5, 6]

La fonction suivante prend en paramètre le programme linéaire dans sa forme standard (donc 3 paramètres : la matrice `A`, le vecteur `b` et le vecteur `c`), et renvoie un couple (tuple à deux éléments) composé des éléments du *tableau* et de la liste des indices des variables basiques.

In [81]:
def create_tableau(
        A: list[list[float]],
        b: list[float],
        c: list[float]) -> tuple[list[list[float]], list[int]]:
    n = len(c)
    m = len(b)
    tableau = [[0] * (n + m + 1) for _ in range(m + 1)]
    for i in range(m):
        for j in range(n):
            tableau[i][j] = A[i][j]
        tableau[i][n + i] = 1
        tableau[i][-1] = b[i]
    for j in range(n):
        tableau[m][j] = c[j]
    basic_vars = list(range(n, n + m+1))
    return tableau, basic_vars

### Choix de la variable entrante

La fonction suivante prend le *tableau* en paramètre et renvoie le numéro de colonne de la variable (non-basique) qui entrera dans la base.

Cette variable est telle que si elle augmente, alors la fonction objectif augmente.

<details>
  <summary>J'ai besoin de plus d'info</summary>
  Les variables qui font augmenter la fonction objectif quand elles augmentent sont celles dont les coefficients sont positifs dans la fonction objectif.
  <div style="margin: 15px;">
  <details>
  <summary>Ok, mais où sont ces coefficients ?</summary>
    En dernière ligne du <i>tableau</i>, en colonne numéro j est stocké le coefficient de la variable numéro j.
  </div>  
</details>

<details>
  <summary>Et s'il y a plusieurs variables comme ça ?</summary>
  Pour éviter les problèmes (ici, pour s'assurer que l'algorithme va bien finir à un moment), une stratégie est de choisir parmi les variables possibles celle dont l'indice est le plus petit.
</details>


In [83]:
def get_entering_var(tableau:list[list[float]])->int|None:
    derniere_ligne=tableau[-1]
    for i in range((0, len(derniere_ligne))-1):
        if derniere_ligne[i] > 0 :
            return i
    return None


### Choix de la variable sortante

Faire entrer une variable dans la base, c'est se "décoller" de l'hyperplan qui la contraignait, parcourir une arête, et s'arrêter lorsque l'on rencontre un autre hyperplan de contrainte (en 2D, hyperplan = droite ; en 3D, hyperplan = plan ; en nD, hyperplan = hyperplan). Et on rencontre un autre hyperplan lorsque l'une des variables qui était dans la base passe à 0. Choisir une variable sortante, c'est trouver une telle variable.

- Dans le *tableau*, les variables candidates pour sortir sont celles qui correspondent à une ligne où la variable entrante est contrainte (*i.e.* une ligne dans laquelle le coefficient de la variable entrante est strictement positif).
- Il peut exister plusieurs variables candidates (lorsqu'on parcourt l'arête, on rencontre un hyperplan de contrainte, puis si on continue un autre hyperplan de contrainte, etc.). On veut rester dans l'ensemble de solutions, donc on ne doit pas passer à travers le premier hyperplan rencontré. Autrement dit, on choisit la variable candidate la plus contrainte.

<details>
<summary>Comment savoir quelle variable candidate est la plus contrainte ?</summary>
Il est sans doute plus facile de se le représenter en revenant à la formulation avec les &le;
  <div style="margin: 15px;">
  <details>
  <summary>Non, je ne vois pas</summary>
    <p>Si on a une contrainte initiale 2 x1 &le; 200 dans la forme canonique, cela nous dit que pour cette contrainte, on peut augmenter x1, mais seulement jusqu'à ce que 2 x1 = 200, c'est-à-dire x1 = 100.
    </p>
    <p> Cette contrainte se traduisait dans la forme standard par exemple par 2x1 + x6 = 200, donc dans la forme matricielle, dans la ligne correspondant à x6, par un coefficient 2 dans la colonne correspondant à x1 et par un coefficient 200 dans la dernière colonne.
    </p>
  </div>  
</details>

La fonction suivante prend le *tableau* et l'indice de la variable entrante en paramètres et renvoie le numéro de ligne de la variable (non-basique) qui entrera dans la base.

In [ ]:
def get_leaving_var_rank(tableau, entering_var):
    indice_du_min=None
    min_trouve_pour_linstant=float('inf')
    for i in range(0, len (tableau)-1):
        if (tableau[i][entering_var] > 0):
            a = tableau[i][-1] / tab[i][entering_var]
            if (a < min_trouve_pour_linstant):
                min_trouve_pour_linstant = a
                indice_du_min = i
    return indice_du_min


### Pivotage de la matrice

La fonction suivante fait pivoter (sur place) le *tableau*, à partir évidemment du *tableau*, du numéro de la colonne de la variable entrante et du numéro de la ligne de la variable sortante.

On peut suivre le plan suivant :
- récupèrer le pivot
- diviser la ligne du pivot par le pivot
- appliquer le principe du pivot de Gauss aux autres lignes : $L_i \leftarrow L_i − \mathit{coeff} \cdot L_{\mathit{pivot}}$ (NB : on a déjà divisé la ligne du pivot par le pivot...)

In [77]:
def pivot(tableau, entering_var, leaving_var_rank):
    return None


### Mise à jour des variables basiques

Si l'on désire seulement obtenir au final la valeur maximale de la fonction objectif, on n'a pas besoin de garder trace des variables basiques. Mais il est souvent intéressant d'avoir également le(s) point(s) de l'espace où la fonction objectif est maximale. Pour cela, on doit se souvenir des échanges entre variables basiques et non-basiques.

La fonction suivante met à jour la liste des variables basiques en fonction du numéro de colonne de la variable entrante et du numéro de ligne de la variable sortante.

In [78]:
def update_basics(basic_vars, leaving_var_rank, entering_var):
    return None


### Arrêt

Quand s'arrête-t-on ?

D'une part, étant donné que les variables sont positives, on peut être sûr que si les coefficients des variables dans la fonction objectif sont tous négatifs, alors une solution où les variables non-basiques sont nulles est optimale.
D'autre part, l'algorithme du simplexe nous assure que l'on parviendra à une telle formulation au bout d'un moment, si le programme linéaire admet une solution "bornée".

La fonction suivante renvoie `True` ssi nous sommes dans un tel cas.

In [79]:
def is_optimal(tableau):
    return None


**TODO** Modifier la fonction `get_leaving_var_rank` pour soulever une exception si le programme linéaire n'admet pas de solution bornée.

### Un simplexe

La fonction suivante est la fonction principale : à partir des données d'un programme linéaire en forme canonique (qui admet une solution nulle), elle implémente un algorithme du simplexe pour renvoyer une solution optimale si elle existe et soulever une exception sinon.

In [80]:
def simplex(A, b, c):
    return None
